# Import Libraries

In [2065]:
#importing the required libraries

import math as m
import datetime as dt
import statistics as stat
import pandas as pd

from pyomo.core import *
from pyomo.environ import *

## You have been given a dataset with the following information

Crushers Location data with their service rate
Diggers Location data with their service rate

You have 3 crushers and 10 diggers on the open pit mine. And you have 130 trucks.
The objective of this model is to build an optimized model such as all loaded trucks are at crushers and all empty ones are at diggers.
The objective is to ensure that no queue is running empty and the queues near the diggers and crushers are optimised so that they can operate efficiently.

##  Task 1: Defining Datastructures: Classes for Truck, Digger and Crusher 

**IMP NOTE:**
In this stub file, we have defined the classes for the trucks, diggers and crushers and for a snapshot that will help you in the truck assignment using the greedy heuristic procedure. We have already explained in detailed comments the steps to be used for every part of the algorithm in the stub file below. To use these classes and instantiate objects related to them as explained during each step in the comments in the part of the stub file below, we suggest you read only that part of OOPs from the link here:_[Link to OOP in Python](https://realpython.com/python3-object-oriented-programming/)_

## Class Truck

In Class Truck, we have attributes as a unique ID for Truck, Source, destination, travel time, mean departure rate and wait time with methods(function that is associated with an object) to update them as well

In [2066]:
class Truck:
    def __init__(self):
        self.global_id = None
        self.source = None
        self.destination = None
        self.travel_time = None
        self.mean_departure_rate = None
        self.wait_time = None
        
    def update_source(self,s):
        self.source = s
        
    def update_destination(self,d):
        self.destination = d
        
    def update_global_id(self, id):
        self.global_id = id
        
    def update_travelTime(self,tme):
        self.travel_time = tme      

# Class Digger

Class Digger has attributes: self-unique ID for Digger, original ID, location, mean and standard deviation time and a list with current queue which will store trucks in the queue for that particular Digger Object.

Digger Class also have methods of updating attributes, updating a truck into the current queue of Digger object and a method to remove all trucks in queue

In [2067]:
class Digger:
    def __init__(self):
        self.id = None
        self.original_id = None
        self.loc_lat_long = []
        self.mean_serviceT = None
        self.std_dev_serviceT = None
        self.current_queue = []
        
    def update_digger(self,lid,oid,lat,lng,meanST,stdST):
        self.id = lid
        self.original_id = oid
        self.loc_lat_long = [lat, lng]
        self.mean_serviceT = meanST
        self.std_dev_serviceT = stdST
    
    # this method appends a single truck to the current_queue list for that particular object
    def update_queue(self,truck_id):
        self.current_queue.append(truck_id)
      
    # below method removes all trucks in queue by updating attribute current queue as empty list
    def remove_allTrucks(self):
        self.current_queue = []

# Class Crusher

Just like Digger class, Class Crusher has attributes: self-unique ID for Crusher, original ID, location, mean and standard deviation time and a list with current queue which will store trucks in the queue for that particular Crusher Object.
Crusher Class also have methods of updating all attribuites, updating a truck into the current queue of Digger object and a method to remove all trucks in queue

In [2068]:
class Crusher:
    def __init__(self):
        self.id = None
        self.original_id = None
        self.loc_lat_long = []
        self.mean_serviceT = None
        self.std_dev_serviceT = None
        self.current_queue = []
        
    def update_crusher(self,lid,oid,lat,lng,meanST,stdST):
        self.id = lid
        self.original_id = oid
        self.loc_lat_long = [lat, lng]
        self.mean_serviceT = meanST
        self.std_dev_serviceT = stdST
        
    # this method appends a single truck to the current_queue list for that particular object
    def update_queue(self,truck_id):
        self.current_queue.append(truck_id)
        
    # below method removes all trucks in queue by updating attribute current queue as empty list
    def remove_allTrucks(self):
        self.current_queue = []

## TASK 2: Reading the input data from CSV files

In [2069]:
# Create and store all the digger and crusher object in following dictionary
digger_objs = {}
crusher_objs = {}

#for each digger, create digger object and store the data into that object and update that object for each digger in digger_objs




#for each crusher, create crusher object and store the data into that object and append that object for each crusher in crusher_objs







## Read all the data from the diggers and crushers location data csv files provided

In [2070]:
#reading the diggers location data in the digger dataframe df_digger
df_digger = pd.read_csv('diggers_location_data.csv') #Read the Digger Location Data here
for ix in df_digger.index:
    d = Digger()
    #updating the attributes in the digger location data for each digger
    d.update_digger(df_digger['SourceLocationName'][ix], df_digger['SourceLocationDescription'][ix],df_digger['Lat'][ix], \
                   df_digger['Lng'][ix], df_digger['Average - Service Time'][ix], df_digger['Std Dev - Service Time'][ix])
    digger_objs[df_digger['SourceLocationName'][ix]] = d
    
diggers = df_digger['SourceLocationName']

#reading the crushers location data in the crusher dataframe df_crusher
df_crusher = pd.read_csv('crushers_location_data.csv') #Read the Crusher Location Data here
for ix in df_crusher.index:
    c = Crusher()
    #updating the attributes in the crusher location data for each crusher
    c.update_crusher(df_crusher['DestinationLocationName'][ix], df_crusher['DestinationLocationDescription'][ix], df_crusher['Lat'][ix], \
                    df_crusher['Lng'][ix], df_crusher['Average - Service Time'][ix], df_crusher['Std Dev - Service Time'][ix])
    crusher_objs[df_crusher['DestinationLocationName'][ix]] = c
    
crushers = df_crusher['DestinationLocationName']

In [2071]:
crusher_objs

{'CR1': <__main__.Crusher at 0x1b3d3981af0>,
 'CR2': <__main__.Crusher at 0x1b3d4f78040>,
 'CR3': <__main__.Crusher at 0x1b3d4f805e0>}

In [2072]:
digger_objs

{'DigLoc1': <__main__.Digger at 0x1b3d4f784f0>,
 'DigLoc2': <__main__.Digger at 0x1b3d4f80c70>,
 'DigLoc3': <__main__.Digger at 0x1b3d4f800a0>,
 'DigLoc4': <__main__.Digger at 0x1b3d4f80a00>,
 'DigLoc5': <__main__.Digger at 0x1b3d4f80e80>,
 'DigLoc6': <__main__.Digger at 0x1b3d4f80850>,
 'DigLoc7': <__main__.Digger at 0x1b3d4f80970>,
 'DigLoc8': <__main__.Digger at 0x1b3d4f80e20>,
 'DigLoc9': <__main__.Digger at 0x1b3d4f802e0>,
 'DigLoc10': <__main__.Digger at 0x1b3d4f803a0>}

In [2073]:
diggers

0     DigLoc1
1     DigLoc2
2     DigLoc3
3     DigLoc4
4     DigLoc5
5     DigLoc6
6     DigLoc7
7     DigLoc8
8     DigLoc9
9    DigLoc10
Name: SourceLocationName, dtype: object

In [2074]:
type(diggers)

pandas.core.series.Series

In [2075]:
diggers[0]

'DigLoc1'

In [2076]:
crushers

0    CR1
1    CR2
2    CR3
Name: DestinationLocationName, dtype: object

# Input data

In [2077]:
n_trucks = 57
n_diggers = len(df_digger)
n_crusher = len(df_crusher)
truck_cap = 230 #tons
avg_truck_speed = 9 #m/s
n_buckets_req_truck = 5

#Index Sets, we are naming Truck ID s T1, T2, T3....
trucks = ['T' + str(x) for x in range(1,n_trucks+1)]
locations = diggers + crushers

#Creating temp 2d array that stores each location's id, mean service time and standard devaition of service time
temp = []
for d in diggers:
    temp.append((d, digger_objs[d].mean_serviceT, digger_objs[d].std_dev_serviceT))
for c in crushers:
    temp.append((c, crusher_objs[c].mean_serviceT, crusher_objs[c].std_dev_serviceT))

    
#Using temp list to store each loaction's mean time and standard devation service time in dictionary
meanServT = {}
stdServT = {}
for item in sorted(temp, key=lambda x:x[1]):
    meanServT[item[0]] = item[1].item() # mapping each loaction ID to mean service time
    stdServT[item[0]] = item[2].item()  # mapping each loaction ID to standard deviation service time

    
#Creating dictinory of truck objects by creating object for each truck and storing it in truck_objs with it's ID as key
truck_objs = {}
for t in range(0,n_trucks):
    truck = Truck()
    truck.update_global_id(trucks[t]) #here Truck ID is T1, T2, T3....
    truck_objs[trucks[t]] = truck

In [2078]:
trucks

['T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19',
 'T20',
 'T21',
 'T22',
 'T23',
 'T24',
 'T25',
 'T26',
 'T27',
 'T28',
 'T29',
 'T30',
 'T31',
 'T32',
 'T33',
 'T34',
 'T35',
 'T36',
 'T37',
 'T38',
 'T39',
 'T40',
 'T41',
 'T42',
 'T43',
 'T44',
 'T45',
 'T46',
 'T47',
 'T48',
 'T49',
 'T50',
 'T51',
 'T52',
 'T53',
 'T54',
 'T55',
 'T56',
 'T57']

In [2079]:
truck_objs

{'T1': <__main__.Truck at 0x1b3d4f80460>,
 'T2': <__main__.Truck at 0x1b3d4f80c10>,
 'T3': <__main__.Truck at 0x1b3d4f80b50>,
 'T4': <__main__.Truck at 0x1b3d4f80a60>,
 'T5': <__main__.Truck at 0x1b3d4f80df0>,
 'T6': <__main__.Truck at 0x1b3d4f80d90>,
 'T7': <__main__.Truck at 0x1b3d4f80760>,
 'T8': <__main__.Truck at 0x1b3d4f808e0>,
 'T9': <__main__.Truck at 0x1b3d4f80c40>,
 'T10': <__main__.Truck at 0x1b3d4f80e50>,
 'T11': <__main__.Truck at 0x1b3d4f807f0>,
 'T12': <__main__.Truck at 0x1b3d4f809a0>,
 'T13': <__main__.Truck at 0x1b3d4f80bb0>,
 'T14': <__main__.Truck at 0x1b3d4f804f0>,
 'T15': <__main__.Truck at 0x1b3d4f80190>,
 'T16': <__main__.Truck at 0x1b3d4f807c0>,
 'T17': <__main__.Truck at 0x1b3d4f80d60>,
 'T18': <__main__.Truck at 0x1b3d4fa5700>,
 'T19': <__main__.Truck at 0x1b3d4fa5880>,
 'T20': <__main__.Truck at 0x1b3d4fa5a60>,
 'T21': <__main__.Truck at 0x1b3d4fa5430>,
 'T22': <__main__.Truck at 0x1b3d4fa52e0>,
 'T23': <__main__.Truck at 0x1b3d4fa5460>,
 'T24': <__main__.Tr

In [2080]:
temp

[('DigLoc1', 19.16, 4.18),
 ('DigLoc2', 25.69, 7.8),
 ('DigLoc3', 28.24, 10.06),
 ('DigLoc4', 20.44, 5.22),
 ('DigLoc5', 36.41, 8.74),
 ('DigLoc6', 40.11, 12.36),
 ('DigLoc7', 29.38, 7.19),
 ('DigLoc8', 31.84, 7.42),
 ('DigLoc9', 23.3, 7.56),
 ('DigLoc10', 39.95, 12.13),
 ('CR1', 60, 120),
 ('CR2', 60, 120),
 ('CR3', 60, 120)]

In [2081]:
trucks

['T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19',
 'T20',
 'T21',
 'T22',
 'T23',
 'T24',
 'T25',
 'T26',
 'T27',
 'T28',
 'T29',
 'T30',
 'T31',
 'T32',
 'T33',
 'T34',
 'T35',
 'T36',
 'T37',
 'T38',
 'T39',
 'T40',
 'T41',
 'T42',
 'T43',
 'T44',
 'T45',
 'T46',
 'T47',
 'T48',
 'T49',
 'T50',
 'T51',
 'T52',
 'T53',
 'T54',
 'T55',
 'T56',
 'T57']

In [2082]:
locations

0    DigLoc1CR1
1    DigLoc2CR2
2    DigLoc3CR3
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
dtype: object

In [2083]:
meanServT

{'DigLoc1': 19.16,
 'DigLoc4': 20.44,
 'DigLoc9': 23.3,
 'DigLoc2': 25.69,
 'DigLoc3': 28.24,
 'DigLoc7': 29.38,
 'DigLoc8': 31.84,
 'DigLoc5': 36.41,
 'DigLoc10': 39.95,
 'DigLoc6': 40.11,
 'CR1': 60,
 'CR2': 60,
 'CR3': 60}

In [2084]:
stdServT

{'DigLoc1': 4.18,
 'DigLoc4': 5.22,
 'DigLoc9': 7.56,
 'DigLoc2': 7.8,
 'DigLoc3': 10.06,
 'DigLoc7': 7.19,
 'DigLoc8': 7.42,
 'DigLoc5': 8.74,
 'DigLoc10': 12.13,
 'DigLoc6': 12.36,
 'CR1': 120,
 'CR2': 120,
 'CR3': 120}

# Calculate Distance with Haversine formula

This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points (ignoring any hills).

Haversine formula:	$a = \sin^2(\Delta \phi /2) + \cos \phi_1 \cdot \cos \phi_2 \cdot \sin^2(\Delta \lambda/2)$

$c = 2 \cdot atan2(\sqrt a, \sqrt{1−a})$

$d = R \cdot c$

where, $\phi$ is latitude, $\lambda$ is longitude, $R$ is earth’s radius (mean radius = 6,371km); note: angles need to be in radians

In [2085]:
def get_geo_distance_calc(lat1, lat2, long1, long2):
    lat1 = lat1 * m.pi / 180
    lat2 = lat2 * m.pi / 180
    long1 = long1 * m.pi / 180
    long2 = long2 * m.pi / 180
    R =  6371 #km
    a = (m.sin((lat2 - lat1)/2.0))**2 + m.cos(lat1) * m.cos(lat2) * (m.sin((long2 - long1)/2.0))**2
    c = 2*m.atan2(m.sqrt(a), m.sqrt(1-a))
    d = R*c
    
    return d

## Calculate distance and truck travel time between each crusher and digger comnbination
truck_travel_time = {}
distance = {}
## distance given in kms
## truck_travel_time = distance * 1000.0 / avg_truck_speed where avg_truck speed is 9 m/s






In [2086]:
#Accessing the locations of digger

digger_objs[diggers[0]].loc_lat_long[0]  

23.37057483

In [2087]:
#Accessing the locations of crusher

crusher_objs[crushers[0]].loc_lat_long[0]

23.38190426

In [2088]:
#calculating the distance between first digger and first crusher

get_geo_distance_calc(digger_objs[diggers[0]].loc_lat_long[0],crusher_objs[crushers[0]].loc_lat_long[0],digger_objs[diggers[0]].loc_lat_long[1],crusher_objs[crushers[0]].loc_lat_long[1])

8.063441408969133

In [2089]:
# Calculating distance between each crusher and digger comnbination and storing into the distance

for i in diggers:
    for j in crushers:
        distance[i+j] = get_geo_distance_calc(digger_objs[i].loc_lat_long[0],crusher_objs[j].loc_lat_long[0],digger_objs[i].loc_lat_long[1],crusher_objs[j].loc_lat_long[1])

In [2090]:
distance # in kms

{'DigLoc1CR1': 8.063441408969133,
 'DigLoc1CR2': 1.4275619623167957,
 'DigLoc1CR3': 14.349318221980422,
 'DigLoc2CR1': 8.112974399564829,
 'DigLoc2CR2': 14.115924217799664,
 'DigLoc2CR3': 1.9051151562176922,
 'DigLoc3CR1': 7.222745451605166,
 'DigLoc3CR2': 11.002065790541453,
 'DigLoc3CR3': 5.746714684518488,
 'DigLoc4CR1': 1.3311659337042046,
 'DigLoc4CR2': 7.916693841533034,
 'DigLoc4CR3': 8.802579082260385,
 'DigLoc5CR1': 2.2611492306240364,
 'DigLoc5CR2': 9.987429107296327,
 'DigLoc5CR3': 5.898362331420833,
 'DigLoc6CR1': 0.8167714364335688,
 'DigLoc6CR2': 7.940267061305039,
 'DigLoc6CR3': 7.278395834103995,
 'DigLoc7CR1': 6.363399326612144,
 'DigLoc7CR2': 1.5288184540450733,
 'DigLoc7CR3': 13.129853872104567,
 'DigLoc8CR1': 5.646188899833118,
 'DigLoc8CR2': 2.5471123351997296,
 'DigLoc8CR3': 12.127517242695431,
 'DigLoc9CR1': 5.2108701163741244,
 'DigLoc9CR2': 3.0005028068533064,
 'DigLoc9CR3': 11.663413134240631,
 'DigLoc10CR1': 0.5498870616399498,
 'DigLoc10CR2': 7.4789788901735

In [2091]:
# Calculating truck travel time between each crusher and digger comnbination and storing into the truck_travel_time dictionary

for i in diggers:
    for j in crushers:
        truck_travel_time[i+j] = (distance[i+j]*1000)/avg_truck_speed
        


In [2092]:
truck_travel_time # in secs

{'DigLoc1CR1': 895.9379343299038,
 'DigLoc1CR2': 158.61799581297728,
 'DigLoc1CR3': 1594.368691331158,
 'DigLoc2CR1': 901.4415999516476,
 'DigLoc2CR2': 1568.4360241999627,
 'DigLoc2CR3': 211.6794618019658,
 'DigLoc3CR1': 802.527272400574,
 'DigLoc3CR2': 1222.4517545046058,
 'DigLoc3CR3': 638.5238538353875,
 'DigLoc4CR1': 147.90732596713383,
 'DigLoc4CR2': 879.632649059226,
 'DigLoc4CR3': 978.0643424733761,
 'DigLoc5CR1': 251.23880340267073,
 'DigLoc5CR2': 1109.7143452551475,
 'DigLoc5CR3': 655.3735923800925,
 'DigLoc6CR1': 90.75238182595209,
 'DigLoc6CR2': 882.2518957005599,
 'DigLoc6CR3': 808.7106482337772,
 'DigLoc7CR1': 707.0443696235716,
 'DigLoc7CR2': 169.86871711611926,
 'DigLoc7CR3': 1458.872652456063,
 'DigLoc8CR1': 627.3543222036798,
 'DigLoc8CR2': 283.0124816888588,
 'DigLoc8CR3': 1347.5019158550479,
 'DigLoc9CR1': 578.9855684860138,
 'DigLoc9CR2': 333.3892007614785,
 'DigLoc9CR3': 1295.9347926934033,
 'DigLoc10CR1': 61.098562404438866,
 'DigLoc10CR2': 830.997654463726,
 'Dig

In [2093]:
#estimating the lambda value from the queue theory using the quadratic equation that we will get above using the values of a,b and c

def get_lambda(a,b,c):
    D = b**2 - 4*a*c
    if D < 0.0:
        print(D)
        return 'infeasible'
    else:
        lambda1 = (-b - m.sqrt(D)) / (2*a)
        lambda2 = (-b + m.sqrt(D)) / (2*a)
        if lambda1 >= 0.0 and lambda2 >= 0.0:
            return [lambda1, lambda2]
        elif lambda1 >= 0.0 and lambda2 < 0.0:
            return [lamdba1]
        elif lambda1 < 0.0 and lambda2 >= 0.0:
            return [lambda2]
        else:
            return 'infeasible'

# Class Snapshot

This class stores following dictionary as attributes:

truck_prevAssign: has dictionary of previous Assignment that maps  digger/crusher to trucks, 

crusher_list: dictionary that stores truck travel time for each crusher, 

digger_list: dictionary that stores truck travel time for each crusher, 

empty_truck_list: dictionary that stores crusher assignment for each empty truck,

loaded_truck_list: dictionary that stores digger assignment for each loaded truck,

queue_len : dictionary that stores queue length for each crusher/digger, 

truck_currentAssign: has dictionary of current Assignment that maps  digger/crusher to trucks(we will print this value),

MF = This calculate matching factor for each crusher/digger

In [2094]:
class Snapshot:
    def __init__(self):
        self.truck_prevAssign = {}
        self.truck_digger_assign_startTime = None
        self.truck_crusher_assign_startTime = None
        self.truck_digger_assign_endTime = None
        self.truck_crusher_assign_endTime = None
        self.crusher_list = {} # maps each crusher to truck_average_time
        self.empty_truck_list = {}
        self.digger_list = {}
        self.loaded_truck_list = {}
        self.start_time = None
        self.queue_len = {}
        #self.cap = {}
        self.truck_currentAssign = {}
        self.MF = {}
        self.n_trucks_crusher_avail = None
        self.n_trucks_digger_avail = None

### Greedy Heuristic Procedure

In [2095]:
#In the below part of the code, all the 4 steps of the algorithm are mentioned in detail. 
#Perform those steps in the same cell and in the order mentioned below.

#snapshots = {}
#time_start = dt.datetime.now()


#you can make 2 snapshots of truck assignment between the diggers and the crushers
#snapshots = {}
#time_start = dt.datetime.now()
#for s in range(0,2): #iterations for each snapshot
 #   print("Snapshot ",s," started\n")
    # snapshot 0:
    # Case 1: with 1 digger and 1 crusher
    # 30 unloaded trucks standing in queue at the digger &
    # 27 loaded trucks standing in queue at the crusher
    
  #  snapshots[s] = Snapshot()    

    # Intialization / starting from previous snapshot
    # assigning the initial positions in case of first snapshot, and previous final assignments in case of subsequent snapshots
    
    
    
    
# create a for loop for a certain number of trucks at each digger and each crusher in the initial snapshot, or else create a new snapshot
# At the start of the schedule, there are 3 trucks each at each digger and 9 trucks each at each crusher


#After having done the above step, please perform the Step 1: Equipment Availability
#Check the status of the trucks in each crusher by checking the following 
#truck.mean_departure_rate
#truck.wait_time
#truck_av_time

#perform the above for each digger and for each crusher


# Step 2: Greedy assignment to estimate rates
# snapshots[s].n_trucks_crusher_avail = 0
# perform the above for every crusher

# snapshots[s].n_trucks_digger_avail = 0
# perform the above for every digger

#now calculate the matching factor
# start by creating an empty list of max_truck_crush_assign_time = []
# Now calculate the truck cycle time and append that to the max_truck_crush_assign_time for every crusher

# Similary, do the above step for diggers as well

# Step 3: Two cases of Truck assignment problem
 #   print("\n Truck Crusher assignment \n")
    
     # Start with the Truck Crusher assignment:
     # create a model construction
#    model = ConcreteModel()

    # define the variables : hint, the variables are related to the available trucks
    # define the objective function and optimise the model cost
    # define the contraints
    # run the model and print the results if it is a feasible solution
    # similarly do the above for the Truck Digger Assignment as well
    # print the snapshots to check the truck assignment

In [2096]:
# Step 1: Estimation of the availability time of crusher, diggers and trucks
#defining the two functions which returns 

# Crushers along with the times when their queues are empty
# Empty trucks at each crusher location, along with the times when they are empty
# Diggers along with the times when their queues are empty
# Loaded trucks at each digger location, along with the times when they are loaded


def Digger_LoadedTruck_List(time_start):
    AvTrucksD={}
    AvDiggers={}
    for i in digger_objs[diggers[0]].current_queue:
        if len(AvTrucksD)==0:
            truck_objs[i].wait_time=0
        else:    
            a = (stdServT[diggers[0]]**2 + meanServT[diggers[0]]**2)/meanServT[diggers[0]]
            b = 2*len(AvTrucksD)*meanServT[diggers[0]]
            c = -2*len(AvTrucksD)
            truck_objs[i].wait_time = len(AvTrucksD)/get_lambda(a,b,c)[0]
        AvTrucksD[i] = time_start.second + truck_objs[i].wait_time + 600
        truck_objs[i].mean_departure_rate = 1/AvTrucksD[i]     
    AvDiggers[diggers[0]] = AvTrucksD[list(AvTrucksD.keys())[-1]]   
    return ([AvTrucksD,AvDiggers])


In [2097]:
def Crusher_EmptyTruck_List(time_start):
    AvTrucksC={}
    Avcrushers={}
    for i in crusher_objs[crushers[0]].current_queue:
        if len(AvTrucksC)==0:
            truck_objs[i].wait_time=0
        else:    
            a = (stdServT[crushers[0]]**2 + meanServT[crushers[0]]**2)/meanServT[crushers[0]]
            b = 2*len(AvTrucksC)*meanServT[crushers[0]]
            c = -2*len(AvTrucksC)
            truck_objs[i].wait_time = len(AvTrucksC)/get_lambda(a,b,c)[0]
        AvTrucksC[i] = time_start.second + truck_objs[i].wait_time + 300
        truck_objs[i].mean_departure_rate = 1/AvTrucksC[i]
    Avcrushers[crushers[0]] = AvTrucksC[list(AvTrucksC.keys())[-1]]   
    return ([AvTrucksC,Avcrushers])

In [2098]:
#creating snapshots with one digger and crusher
# 30 unloaded trucks standing in queue at the digger &
# 27 loaded trucks standing in queue at the crusher

snapshots = {}
unloaded = 30
loaded = 27
startD=0
startC=unloaded
time_start = dt.datetime.now()   # starting the time 

for s in range(0,2):
    print("Snapshot ",s," started\n")

    time_start = dt.datetime.now() # starting the time of each snapshot

    snapshots[s] = Snapshot()    
    snapshots[s].start_time=time_start
    snapshots[s].truck_digger_assign_startTime=time_start
    snapshots[s].truck_crusher_assign_startTime=time_start
    
    snapshots[s].truck_prevAssign[diggers[0]] = None
    snapshots[s].truck_prevAssign[crushers[0]]  = None
    snapshots[s].truck_currentAssign[diggers[0]] = None
    snapshots[s].truck_currentAssign[crushers[0]]  = None
    
    # Intialization / starting from previous snapshot
    # assigning the initial positions in case of first snapshot, and previous final assignments in case of subsequent snapshots
    
    
    if s!=0:
        snapshots[s].truck_prevAssign[diggers[0]] = snapshots[s-1].truck_currentAssign[diggers[0]]  
        snapshots[s].truck_prevAssign[crushers[0]] = snapshots[s-1].truck_currentAssign[crushers[0]]
        
        loaded = len(snapshots[s-1].truck_currentAssign[diggers[0]])  #30
        startD = loaded
        unloaded = len(snapshots[s-1].truck_currentAssign[crushers[0]]) #27
        startC=0
        
        
    snapshots[s].loaded_truck_list[diggers[0]] = trucks[:unloaded]
    
    for t in range(startD,unloaded+startD):
        digger_objs[diggers[0]].update_queue(trucks[t])
        truck_objs[trucks[t]].update_source(diggers[0])
        truck_objs[trucks[t]].update_destination(crushers[0])
        truck_objs[trucks[t]].update_travelTime(truck_travel_time[diggers[0]+crushers[0]])
    
        
        snapshots[s].digger_list[trucks[t]] = truck_travel_time[diggers[0]+crushers[0]]
    
    snapshots[s].empty_truck_list[crushers[0]] = trucks[unloaded:unloaded+loaded]
    
    for c in range(startC,startC+loaded):
        crusher_objs[crushers[0]].update_queue(trucks[c])
        truck_objs[trucks[c]].update_source(crushers[0])
        truck_objs[trucks[c]].update_destination(diggers[0])
        truck_objs[trucks[c]].update_travelTime(truck_travel_time[diggers[0]+crushers[0]])
    
        snapshots[s].crusher_list[trucks[t]] = truck_travel_time[diggers[0]+crushers[0]]
    

    
    time_end = dt.datetime.now()
    snapshots[s].truck_digger_assign_endTime=time_end
    snapshots[s].truck_crusher_assign_endTime=time_end
    
    
    if s==0:    
        snapshots[s].truck_currentAssign[diggers[0]] = digger_objs[diggers[0]].current_queue
        snapshots[s].truck_currentAssign[crushers[0]] = crusher_objs[crushers[0]].current_queue
    else:
        snapshots[s].truck_currentAssign[diggers[0]] = crusher_objs[crushers[0]].current_queue
        snapshots[s].truck_currentAssign[crushers[0]] =digger_objs[diggers[0]].current_queue

    
    
    # step 1 : Equipment availability
    
    Digger_LoadedTruck_List(snapshots[s].truck_digger_assign_startTime)
    Crusher_EmptyTruck_List(snapshots[s].truck_crusher_assign_startTime)
    
    
    
    
    snapshots[s].queue_len[diggers[0]] = len(digger_objs[diggers[0]].current_queue)
    snapshots[s].queue_len[crushers[0]] = len(crusher_objs[crushers[0]].current_queue)
    
    
    
    
    
    
    # Step 2: rate estimation of diggers and crushers
    
    snapshots[s].n_trucks_crusher_avail = len(crusher_objs[crushers[0]].current_queue)
    snapshots[s].n_trucks_digger_avail = len(digger_objs[diggers[0]].current_queue)
    
    diggerRate = {diggers[0]+'Rate':len(digger_objs[diggers[0]].current_queue)*truck_cap}
    crusherRate = {crushers[0]+'Rate':len(crusher_objs[crushers[0]].current_queue)*truck_cap}    
    
    # Step 3: calculating matching factor
    
    digger_truck = Digger_LoadedTruck_List(snapshots[s].truck_digger_assign_startTime)[0]
    CrusherAvailableTime = Crusher_EmptyTruck_List(snapshots[s].truck_crusher_assign_startTime)[1]
    #print(digger_truck)
    MFCrusher={}
    for i in digger_truck:
        if i == list(digger_truck)[0]:
            time_start = digger_truck[i]
        arrivalTime = digger_truck[i] + truck_travel_time[diggers[0]+crushers[0]] - time_start
        TruckCycleTime = arrivalTime
        MFCrusher['MFCrusher_'+i] = (loaded*CrusherAvailableTime[crushers[0]])/(1*TruckCycleTime)
    #print(MFCrusher)
    
    crusher_truck = Crusher_EmptyTruck_List(snapshots[s].truck_crusher_assign_startTime)[0]
    DiggerAvailableTime = Digger_LoadedTruck_List(snapshots[s].truck_digger_assign_startTime)[1]
    #print(digger_truck)
    MFDigger={}
    for i in crusher_truck:
        if i == list(crusher_truck)[0]:
            time_start = crusher_truck[i]
        arrivalTime = crusher_truck[i] + truck_travel_time[diggers[0]+crushers[0]] - time_start
        TruckCycleTime = arrivalTime
        MFDigger['MFDigger_'+i] = (loaded*DiggerAvailableTime[diggers[0]])/(1*TruckCycleTime)
    #print(MFDigger)
    
    
        
        
            
            
    
    #print(digger_objs[diggers[0]].current_queue)
    print(snapshots[s].truck_currentAssign)

    digger_objs[diggers[0]].remove_allTrucks()
    crusher_objs[crushers[0]].remove_allTrucks()
    
    #print(snapshots[s].truck_currentAssign)
    #print(snapshots[s].truck_prevAssign)
    #print(diggerRate)
    #print(crusherRate)




Snapshot  0  started

{'DigLoc1': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T30'], 'CR1': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39', 'T40', 'T41', 'T42', 'T43', 'T44', 'T45', 'T46', 'T47', 'T48', 'T49', 'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57']}
Snapshot  1  started

{'DigLoc1': ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T30'], 'CR1': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39', 'T40', 'T41', 'T42', 'T43', 'T44', 'T45', 'T46', 'T47', 'T48', 'T49', 'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57']}


In [2099]:
# Step 1: Estimation of the availability time of crusher, diggers and trucks
#defining the two functions which returns 

# Crushers along with the times when their queues are empty
# Empty trucks at each crusher location, along with the times when they are empty
# Diggers along with the times when their queues are empty
# Loaded trucks at each digger location, along with the times when they are loaded


def Digger_LoadedTruck_List2(time_start):
    AvTrucksD={}
    AvDiggers={}
    for d in range(10):
        for i in digger_objs[diggers[d]].current_queue:
            if len(AvTrucksD)==0:
                truck_objs[i].wait_time=0
            else:    
                a = (stdServT[diggers[d]]**2 + meanServT[diggers[d]]**2)/meanServT[diggers[d]]
                b = 2*len(AvTrucksD)*meanServT[diggers[d]]
                c = -2*len(AvTrucksD)
                truck_objs[i].wait_time = len(AvTrucksD)/get_lambda(a,b,c)[0]
            AvTrucksD[i] = time_start.second + truck_objs[i].wait_time + 600
            truck_objs[i].mean_departure_rate = 1/AvTrucksD[i]     
        AvDiggers[diggers[d]] = AvTrucksD[list(AvTrucksD.keys())[-1]]   
    return ([AvTrucksD,AvDiggers])


In [2100]:
def Crusher_EmptyTruck_List2(time_start):
    AvTrucksC={}
    Avcrushers={}
    for k in range(3):
        for i in crusher_objs[crushers[k]].current_queue:
            if len(AvTrucksC)==0:
                truck_objs[i].wait_time=0
            else:    
                a = (stdServT[crushers[k]]**2 + meanServT[crushers[k]]**2)/meanServT[crushers[k]]
                b = 2*len(AvTrucksC)*meanServT[crushers[k]]
                c = -2*len(AvTrucksC)
                truck_objs[i].wait_time = len(AvTrucksC)/get_lambda(a,b,c)[0]
            AvTrucksC[i] = time_start.second + truck_objs[i].wait_time + 300
            truck_objs[i].mean_departure_rate = 1/AvTrucksC[i]
        Avcrushers[crushers[k]] = AvTrucksC[list(AvTrucksC.keys())[-1]]   
    return ([AvTrucksC,Avcrushers])

In [2101]:
# now we are considering all crushers and diggers
# create a for loop for a certain number of trucks at each digger and each crusher in the initial snapshot, or else create a new snapshot
# At the start of the schedule, there are 3 trucks each at each digger and 9 trucks each at each crusher


#creating snapshots with one digger and crusher


snapshots = {}

time_start = dt.datetime.now()   # starting the time 

for s in range(0,2):
    print("\nSnapshot ",s," started\n")

    time_start = dt.datetime.now() # starting the time of each snapshot

    snapshots[s] = Snapshot()    
    snapshots[s].start_time=time_start
    snapshots[s].truck_digger_assign_startTime=time_start
    snapshots[s].truck_crusher_assign_startTime=time_start
    
    snapshots[s].truck_prevAssign[diggers[0]] = None
    snapshots[s].truck_prevAssign[crushers[0]]  = None
    snapshots[s].truck_currentAssign[diggers[0]] = None
    snapshots[s].truck_currentAssign[crushers[0]]  = None
    
    # Intialization / starting from previous snapshot
    # assigning the initial positions in case of first snapshot, and previous final assignments in case of subsequent snapshots
    
    
    #if s!=0:
     #   snapshots[s].truck_prevAssign[diggers[0]] = snapshots[s-1].truck_currentAssign[diggers[0]]  
     #   snapshots[s].truck_prevAssign[crushers[0]] = snapshots[s-1].truck_currentAssign[crushers[0]]
        
      #  loaded = len(snapshots[s-1].truck_currentAssign[diggers[0]])  #30
       # startD = loaded
        #unloaded = len(snapshots[s-1].truck_currentAssign[crushers[0]]) #27
        #startC=0
        
        
    #snapshots[s].loaded_truck_list[diggers[0]] = trucks[:unloaded]
    
    #initializing three trucks at each digger
    
    x=0
    for d in range(10):
        for t in range(3):
            digger_objs[diggers[d]].update_queue(trucks[x])
            truck_objs[trucks[x]].update_source(diggers[d])
            x+=1
        #print(digger_objs[diggers[d]].current_queue)
    
    #initializing nine trucks at each crusher
    

    for c in range(3):
        for t in range(9):
            crusher_objs[crushers[c]].update_queue(trucks[x])
            truck_objs[trucks[x]].update_source(crushers[c])
            x+=1
        #print(crusher_objs[crushers[c]].current_queue)

        
        
        
    #snapshots[s].digger_list[trucks[t]] = truck_travel_time[diggers[0]+crushers[0]]
    
    #snapshots[s].empty_truck_list[crushers[0]] = trucks[unloaded:unloaded+loaded]
    
    
    #snapshots[s].crusher_list[trucks[t]] = truck_travel_time[diggers[0]+crushers[0]]
    

    
    time_end = dt.datetime.now()
    snapshots[s].truck_digger_assign_endTime=time_end
    snapshots[s].truck_crusher_assign_endTime=time_end
    
    
    if s==0:
        for i in range(10):
            snapshots[s].truck_currentAssign[diggers[i]] = digger_objs[diggers[i]].current_queue
        for c in range(3):
            snapshots[s].truck_currentAssign[crushers[c]] = crusher_objs[crushers[c]].current_queue
    else:
        snapshots[s].truck_currentAssign[diggers[0]] = crusher_objs[crushers[0]].current_queue
        snapshots[s].truck_currentAssign[crushers[0]] =digger_objs[diggers[0]].current_queue

    
    
    # step 1 : Equipment availability
    
    #print(Digger_LoadedTruck_List2(snapshots[s].truck_digger_assign_startTime))
    #print(Crusher_EmptyTruck_List2(snapshots[s].truck_crusher_assign_startTime))
    
    
    
    
    #snapshots[s].queue_len[diggers[0]] = len(digger_objs[diggers[0]].current_queue)
    #snapshots[s].queue_len[crushers[0]] = len(crusher_objs[crushers[0]].current_queue)
    
    
    
    
    
    
    # Step 2: rate estimation of diggers and crushers
    
    
    #snapshots[s].n_trucks_crusher_avail = len(crusher_objs[crushers[0]].current_queue)
    #snapshots[s].n_trucks_digger_avail = len(digger_objs[diggers[0]].current_queue)
    
    crusherRate={}
    for k in range(3):
        crusherRate[crushers[k]+'Rate'] = len(crusher_objs[crushers[k]].current_queue)*truck_cap
    #print(crusherRate)
    
    diggerRate={}
    for d in range(10):
        diggerRate[diggers[d]+'Rate'] = len(digger_objs[diggers[d]].current_queue)*truck_cap
    #print(diggerRate)
    
    # Step 3: calculating matching factor
    
    digger_truck = Digger_LoadedTruck_List2(snapshots[s].truck_digger_assign_startTime)[0]
    CrusherAvailableTime = Crusher_EmptyTruck_List2(snapshots[s].truck_crusher_assign_startTime)[1]
    #print(digger_truck)
    
    MFCrusher={}
    for k in CrusherAvailableTime:
        for t in digger_truck:
            if t == list(digger_truck)[0]:
                time_start = digger_truck[t]
            arrivalTime = digger_truck[t] + truck_travel_time[truck_objs[t].source+k] - time_start
            TruckCycleTime = arrivalTime
            MFCrusher['MFCrusher_'+k] = (loaded*CrusherAvailableTime[k])/(1*TruckCycleTime)
    #print(MFCrusher)
    
    crusher_truck = Crusher_EmptyTruck_List2(snapshots[s].truck_crusher_assign_startTime)[0]
    DiggerAvailableTime = Digger_LoadedTruck_List2(snapshots[s].truck_digger_assign_startTime)[1]
    
    #print(digger_truck)
    
    MFDigger={}
    for d in DiggerAvailableTime: 
        for t in crusher_truck:
            if t == list(crusher_truck)[0]:
                time_start = crusher_truck[t]
            arrivalTime = crusher_truck[t] + truck_travel_time[d+truck_objs[t].source] - time_start
            TruckCycleTime = arrivalTime
            MFDigger['MFDigger_'+d] = (loaded*DiggerAvailableTime[d])/(1*TruckCycleTime)
    #print(MFDigger)
    
    
        
        
            
            
    
    #print(digger_objs[diggers[0]].current_queue)
    for d in range(10):
        digger_objs[diggers[d]].remove_allTrucks()
    for c in range(3):
        crusher_objs[crushers[c]].remove_allTrucks()
    
    print(snapshots[s].truck_currentAssign)
    #print(snapshots[s].truck_prevAssign)
    #print(diggerRate)
    #print(crusherRate)





Snapshot  0  started

{'DigLoc1': ['T1', 'T2', 'T3'], 'CR1': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39'], 'DigLoc2': ['T4', 'T5', 'T6'], 'DigLoc3': ['T7', 'T8', 'T9'], 'DigLoc4': ['T10', 'T11', 'T12'], 'DigLoc5': ['T13', 'T14', 'T15'], 'DigLoc6': ['T16', 'T17', 'T18'], 'DigLoc7': ['T19', 'T20', 'T21'], 'DigLoc8': ['T22', 'T23', 'T24'], 'DigLoc9': ['T25', 'T26', 'T27'], 'DigLoc10': ['T28', 'T29', 'T30'], 'CR2': ['T40', 'T41', 'T42', 'T43', 'T44', 'T45', 'T46', 'T47', 'T48'], 'CR3': ['T49', 'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57']}

Snapshot  1  started

{'DigLoc1': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39'], 'CR1': ['T1', 'T2', 'T3']}
